In [197]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers

pd.set_option('display.max_rows', 500)

In [119]:
def print_score(model, x, y):
    print('Score:        {:.4f}%'.format(model.score(x, y) * 100))
    print('Kaggle Score: {:.0f}'.format(np.sqrt(np.mean((model.predict(x) - y)**2))))   

In [206]:
# Cleaning Data

df = pd.read_csv('data/stock_XY_train.csv')
print('OG Data Size:{}'.format(df.shape))

df = df[df.columns[df.isnull().mean() < 0.25]] # TO-DO: Tinker around with mean threshold.
df = df.dropna()
print('New Data Size:{}'.format(df.shape))
del df['operatingProfitMargin'] # Got rid of this column because it is all `1`. No reason to keep.

pd.get_dummies(df, columns = ['Sector', 'Ticker'])
print(list(df.columns))
# print(df)
# df.to_csv('new_data.csv', sep=',')
# df.isna().sum()

OG Data Size:(12379, 226)
New Data Size:(6030, 188)
['Unnamed: 0', 'Ticker', 'Revenue', 'Revenue Growth', 'Cost of Revenue', 'Gross Profit', 'R&D Expenses', 'SG&A Expense', 'Operating Expenses', 'Operating Income', 'Interest Expense', 'Earnings before Tax', 'Income Tax Expense', 'Net Income - Non-Controlling int', 'Net Income - Discontinued ops', 'Net Income', 'Preferred Dividends', 'Net Income Com', 'EPS', 'EPS Diluted', 'Weighted Average Shs Out', 'Weighted Average Shs Out (Dil)', 'Dividend per Share', 'Gross Margin', 'EBITDA Margin', 'EBIT Margin', 'Profit Margin', 'Free Cash Flow margin', 'EBITDA', 'EBIT', 'Consolidated Income', 'Earnings Before Tax Margin', 'Net Profit Margin', 'Cash and cash equivalents', 'Short-term investments', 'Cash and short-term investments', 'Receivables', 'Inventories', 'Total current assets', 'Property, Plant & Equipment Net', 'Goodwill and Intangible Assets', 'Long-term investments', 'Tax assets', 'Total assets', 'Payables', 'Short-term debt', 'Total cu

In [171]:
# Separating to train and test data
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

# Grabbing stats in order to normalize data
train_stats = train_df.describe()
train_stats.pop('Buy')
train_stats = train_stats.transpose()

# Separating labels
train_label = train_df.pop('Buy')
test_label = test_df.pop('Buy')

# Normalizing Data
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

temp_train = train_df[['Ticker','Sector']].copy()
# print(temp_train['Yr'])
normed_train_data = norm(train_df[train_df.columns.difference(['Ticker','Sector'])])
normed_train_data = pd.concat([temp_train, normed_train_data], axis=1)

temp_test = test_df[['Ticker','Sector']].copy()
normed_test_data = norm(test_df[test_df.columns.difference(['Ticker','Sector'])])
normed_test_data = pd.concat([temp_test, normed_test_data], axis=1)

In [162]:
def build_model():
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=[len(train_df.keys())]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
    return model

model = build_model()
model.summary()

example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 64)                11968     
_________________________________________________________________
dense_19 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 65        
Total params: 16,193
Trainable params: 16,193
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking input: expected dense_18_input to have shape (186,) but got array with shape (188,)